## Import

In [1]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
DIR_PATH = "gdrive/MyDrive/ChamoAnalytics"

In [4]:
df = pd.read_csv(f"{DIR_PATH}/master_dataset.csv", na_values=['-1'])

## Preprocessing

In [5]:
df = df.replace('.', np.nan)
df[[" us_bank_rate"]] = df[[" us_bank_rate"]].apply(pd.to_numeric)
df = df.rename(columns={' us_bank_rate': 'us_bank_rate'})

In [6]:
df.dtypes

date                            object
rate                           float64
sent_fin_us_uncertainty        float64
sent_fin_us_litigious          float64
sent_fin_us_strong_modal       float64
sent_fin_us_weak_modal         float64
sent_fin_us_constraining       float64
sent_fin_us_optimistic         float64
sent_fin_can_uncertainty       float64
sent_fin_can_litigious         float64
sent_fin_can_strong_modal      float64
sent_fin_can_weak_modal        float64
sent_fin_can_constraining      float64
sent_fin_can_optimistic        float64
ti_rate_rsi_14                 float64
ti_rate_stochrsi_14            float64
ti_rate_macd_12_26             float64
ti_rate_adx_14                 float64
ti_rate_williams_%R            float64
ti_rate_cci                    float64
ti_rate_atr                    float64
ti_rate_utlimate_oscillator    float64
ti_rate_roc                    float64
interest_rate_can              float64
index_W.BCPI                   float64
index_W.BCNE             

In [7]:
df.size

1028250

In [8]:
df['y_exp'] = np.full(df.shape[0], -1)

for i, row in df.iterrows():
    try:
        two_weeks_from_current_date = datetime.datetime.strptime(row.date, "%Y-%m-%d") + datetime.timedelta(weeks=2)

        futur_rate = list(df.loc[df['date'] == str(two_weeks_from_current_date.date())].rate)[0]
        df.loc[i, 'y_exp' ] = futur_rate
    except: 
        break

df = df.loc[df.y_exp >= 0]

In [9]:
df = df.dropna()
df = df.reset_index(drop=True)

In [10]:
df.size

307924

In [11]:
def to_sequences(seq_size, obs, y):
    x = []
    y_out = []

    for i in range(len(obs)-seq_size):
        #print(i)
        window = obs[i:(i+seq_size)]
        after_window = y[i+seq_size - 1]
        window = [x for x in window]
        x.append(window)
        y_out.append(after_window)
        
    return np.array(x), np.array(y_out)

In [12]:
def split_train_test_val(df):
    # Split train val test datasets
    test_cutting_date = '2022-05-01'
    val_cutting_date = '2022-02-01'
    df_train = df[df['date'] < val_cutting_date]
    df_val = df[(df['date'] >= val_cutting_date) &
                (df['date'] < test_cutting_date)]
    df_test = df[df['date'] >= test_cutting_date]
    
    return df_train, df_val, df_test

In [13]:
def prepare_data(df, cols_to_keep, seq_size):
    df_train, df_val, df_test = split_train_test_val(df)


    x_train = df_train[cols_to_keep].to_numpy()
    y_train = df_train['y_exp'].to_numpy()

    x_val = df_val[cols_to_keep].to_numpy()
    y_val = df_val['y_exp'].to_numpy()

    x_test = df_test[cols_to_keep].to_numpy()
    y_test = df_test['y_exp'].to_numpy()

    x_train, y_train = to_sequences(seq_size, x_train, y_train)
    x_val, y_val = to_sequences(seq_size, x_val, y_val)
    x_test, y_test = to_sequences(seq_size, x_test, y_test)

    print("Train: {}".format(x_train.shape))
    print("Val: {}".format(x_val.shape))
    print("Test: {}".format(x_test.shape))

    return x_train, y_train, x_val, y_val, x_test, y_test

## Modeling

In [14]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)

    return x + res

In [15]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    
    return keras.Model(inputs, outputs)

## Training

In [16]:
def train(x_train, y_train, x_val, y_val):
    input_shape = x_train.shape[1:]

    model = build_model(
        input_shape,
        head_size=256,
        num_heads=4,
        ff_dim=4,
        num_transformer_blocks=4,
        mlp_units=[128],
        mlp_dropout=0.4,
        dropout=0.25,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=1e-4)
    )

    callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

    model.fit(
        x=x_train,
        y=y_train,
        validation_data=(x_val, y_val),
        epochs=200,
        batch_size=64,
        callbacks=callbacks,
    )

    return model

## Evaluating

In [17]:
def run(df, cols_to_keep, seq_size, nb_trials=5):
    x_train, y_train, x_val, y_val, x_test, y_test = prepare_data(df, cols_to_keep, seq_size)

    errors = []
    models = []
    smallest_error = None

    for i in range(nb_trials):
        model = train(x_train, y_train, x_val, y_val)
        error = model.evaluate(x_test, y_test, verbose=1)

        if not smallest_error or smallest_error > error:
            smallest_error = error

        errors.append(error)
        models.append(model)
    
    print('mean: ', np.mean(errors), 'var: ', np.var(errors))
    print('smallest error: ', smallest_error)

    return models

### Top-5 of the features based on LSTM results

In [18]:
cols_to_keep = [
    'interest_rate_can',
    'index_WGTS.BRENT',
    'rate',
    'ti_rate_rsi_14',
    'sent_fin_can_litigious',
]

seq_size = 20
models = run(df, cols_to_keep, seq_size, nb_trials=5)

Train: (6490, 20, 5)
Val: (43, 20, 5)
Test: (101, 20, 5)
Epoch 1/200
102/102 [==============================] - 13s 22ms/step - loss: 21.3917 - val_loss: 0.3878
Epoch 2/200
102/102 [==============================] - 1s 14ms/step - loss: 5.1756 - val_loss: 0.8645
Epoch 3/200
102/102 [==============================] - 1s 15ms/step - loss: 2.1757 - val_loss: 0.5818
Epoch 4/200
102/102 [==============================] - 1s 15ms/step - loss: 1.6844 - val_loss: 0.4575
Epoch 5/200
102/102 [==============================] - 2s 15ms/step - loss: 1.5196 - val_loss: 0.4084
Epoch 6/200
102/102 [==============================] - 1s 15ms/step - loss: 1.3752 - val_loss: 0.3268
Epoch 7/200
102/102 [==============================] - 1s 15ms/step - loss: 1.2270 - val_loss: 0.3371
Epoch 8/200
102/102 [==============================] - 2s 15ms/step - loss: 1.1316 - val_loss: 0.2692
Epoch 9/200
102/102 [==============================] - 2s 15ms/step - loss: 1.0430 - val_loss: 0.2391
Epoch 10/200
102/102 [=

In [19]:
for i, model in enumerate(models):
    model.save(f"{DIR_PATH}/transformer_models/model_{i}")